## Most Popular Client For Calls - StrataScratch

Select the most popular client_id based on the number of users who individually have at least 50% of their events from the following list: 'video call received', 'video call sent', 'voice call received', 'voice call sent'.

Table: fact_events

In [13]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import pandas as pd

import os
import sys

In [17]:
os.environ["JAVA_HOME"] = "C:/Program Files/Java/jdk-11"
spark = SparkSession.builder.appName("Most Popular Client For Calls").getOrCreate()

In [18]:
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("../Data/Most Popular Client For Calls_fact_events_Tbl.csv")

In [19]:
df.show() #only shows 20 rows
# df.toPandas() # will not work as it requires 3.11 python version or lower

+---+-------+----------+----------------+---------+-------------------+--------+
| id|time_id|   user_id|     customer_id|client_id|         event_type|event_id|
+---+-------+----------+----------------+---------+-------------------+--------+
|  1|  43889|3668-QPYBK|          Sendit|  desktop|       message sent|       3|
|  2|  43889|7892-POOKP|       Connectix|   mobile|      file received|       2|
|  3|  43924|9763-GRSKD|          Zoomit|  desktop|video call received|       7|
|  4|  43923|9763-GRSKD|       Connectix|  desktop|video call received|       7|
|  5|  43867|9237-HQITU|          Sendit|  desktop|video call received|       7|
|  6|  43888|8191-XWSZG|       Connectix|  desktop|      file received|       2|
|  7|  43924|9237-HQITU|       Connectix|  desktop|video call received|       7|
|  8|  43891|9237-HQITU|       Connectix|   mobile|   message received|       4|
|  9|  43923|4190-MFLUW|       Connectix|   mobile|video call received|       7|
| 10|  43942|9763-GRSKD|    